In [1]:
import tensorflow as tf
import cv2
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
import glob
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
tf.__version__

'2.1.0'

In [ ]:
def inner_product(left, right, disp, width):
    
    disp_vol = []
  
    for i in range(disp):
        output = tf.reduce_sum(tf.keras.layers.multiply([left, right[:,disp-i-1:disp -1 -i + width,:,: ]]), axis = -1)
        disp_vol.append(output)
    logits = tf.transpose(tf.stack(disp_vol),[1,2,3,0])
    logits = tf.nn.softmax(logits)
    return logits

In [ ]:
def valid_pixels(y, y_gt,num_classes):
    valid_pixels = tf.not_equal(y_gt, 0)
    labels = tf.reshape(tf.boolean_mask(y_gt, valid_pixels),[-1,num_classes])
    logits = tf.reshape(tf.boolean_mask(y, valid_pixels),[-1])
    return labels, logits


In [ ]:
def disparity_threshold(gt, max_disp):
    return gt * (gt <= max_disp)
    

In [ ]:
def create_training_dataset(path, patch_size, max_disp):
    %matplotlib
    gt_path = sorted(glob.glob(path+'KITTI_dataset/2015/training/disp_noc_0/*.png'))
    left_image = sorted(glob.glob(path+'KITTI_dataset/2015/training/image_2/*.png'))
    right_image = sorted(glob.glob(path+'KITTI_dataset/2015/training/image_3/*.png'))
    gt_patches = []
    left_patches = []
    right_patches = []
    
    
    for i in range(len(gt_path)):
        print('Progress: ','{:.5}'.format( str(i / (len(gt_path)-1) * 100)),'%',end='\r')
        assert(gt_path[i].split('/')[-1] == left_image[i].split('/')[-1])
        assert(gt_path[i].split('/')[-1] == right_image[i].split('/')[-1])

        gt = cv2.imread(gt_path[i], cv2.IMREAD_GRAYSCALE)
        gt = disparity_threshold(gt, max_disp)
        left = cv2.imread(left_image[i],cv2.IMREAD_COLOR)
        left = cv2.cvtColor(left,cv2.COLOR_BGR2RGB)
        right = cv2.imread(right_image[i], cv2.IMREAD_COLOR)
        right = cv2.cvtColor(right, cv2.COLOR_BGR2RGB)
       # right_padding = np.zeros((patch_size,(patch_size+max_disp ) ,3),dtype = 'uint8') #convert into same format uint8

        for b in range(int(left.shape[0]/patch_size)):
            for a in range(int(left.shape[1]/patch_size)):
                right_padding = np.zeros((patch_size,(patch_size+max_disp ) ,3),dtype = 'uint8') #convert into same format uint8
                x = a * patch_size
                y = b * patch_size

                left_patches.append(left[y:patch_size+y,x:(x+patch_size),:])
                gt_patches.append(gt[y:patch_size+y,x: (x+patch_size)])

                if (x - max_disp) < 0:
                    right_patch = right[y:patch_size+y, 0 : (x  + patch_size),:]
                    right_padding[:,(max_disp + patch_size) - right_patch.shape[1]:,:] = right_patch
                    right_patches.append(right_padding)
                else:
                    right_patch = right[y:patch_size+y, (x - max_disp)  : (x   + patch_size),:]
                    right_patches.append(right_patch)
    

    num_seg =5
    seg = int(np.array(left_patches).shape[0] / num_seg )
    print('\r')
    print('Segmentation Completed')
    for i in range(num_seg):
        print("Saving:",'{:.5}'.format((i / (num_seg-1))*100),end ='\r')
        x = i * seg
        np.save(path+'Data/training_data/left_patches_'+str(i)+'.npy', np.array(left_patches[x:x+seg]))
        np.save(path+'Data/training_data/right_patches_'+str(i)+'.npy', np.array(right_patches[x:x+seg]))
        np.save(path+'Data/training_data/gt_'+str(i)+'.npy', np.array(gt_patches[x:x+seg]))
        

# Creating Keras Model 

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters = 64, kernel_size=(3,3) , strides = (1,1), padding = "same",kernel_initializer='glorot_uniform'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3) , strides = (1,1) , padding = "same" , kernel_initializer = 'glorot_uniform'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())

model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))

model.add(keras.layers.Conv2D(filters = 64, kernel_size=(3,3) , strides = (1,1), padding = "same",kernel_initializer='glorot_uniform'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3) , strides = (1,1) , padding = "same" , kernel_initializer = 'glorot_uniform'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())

model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))

model.add(keras.layers.Conv2D(filters = 64, kernel_size=(3,3) , strides = (1,1), padding = "same",kernel_initializer='glorot_uniform'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3) , strides = (1,1) , padding = "same" , kernel_initializer = 'glorot_uniform'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.ReLU())

model.add(keras.layers.Conv2D(filters = 64, kernel_size=(3,3) , strides = (1,1), padding = "same",kernel_initializer='glorot_uniform'))
model.add(keras.layers.Conv2DTranspose(filters = 64, kernel_size = (2,2) , strides = (2,2), kernel_initializer = 'glorot_uniform'))
model.add(keras.layers.Conv2DTranspose(filters = 64, kernel_size = (2,2) , strides = (2,2), kernel_initializer = 'glorot_uniform'))


In [49]:
path = '/media/sansii/Software/san_projects/Major_project/'
create_training_dataset(path, 28, 128)

Using matplotlib backend: TkAgg
Progress:  100.0 %
Segmentation Completed


In [50]:
path = '/media/sansii/Software/san_projects/Major_project/'
left = np.load(path+"Data/training_data/left_patches_0.npy")
right = np.load(path+"Data/training_data/right_patches_0.npy")
gt = np.load(path+"Data/training_data/gt_0.npy")

In [52]:
%matplotlib
a = 47
plt.subplot(2,2,1)
plt.imshow(left[a])
plt.subplot(2,2,2)

plt.imshow(right[a])
plt.subplot(2,2,3)
plt.imshow(gt[a],cmap = 'jet')
plt.subplot(2,2,4)
plt.show()
print(left.shape)

Using matplotlib backend: TkAgg
(22874, 28, 28, 3)


In [ ]:
max_disp = 128
W_l = 28
W_r = 28+max_disp
H = 28

input_left =  keras.Input(shape = (W_l, H, 3))
input_right = keras.Input(shape = (W_r, H ,3))

output_left = model(input_left)
output_right= model(input_right)

outputs = inner_product(output_left, output_right, max_disp + 1, W_l)


siamese_model = keras.models.Model([input_left, input_right], outputs)
siamese_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      optimizer=keras.optimizers.Adam(learning_rate=1e-6), metrics=['accuracy'])

In [ ]:
siamese_model.summary()

In [ ]:
from PIL import Image
import numpy as np
%matplotlib
path = '/media/sansii/Software/san_projects/Major_project/KITTI_dataset/training/disp_noc_0'
img = cv2.imread(path+'000023_10.png',cv2.IMREAD_GRAYSCALE)
img = np.array(img)
plt.figure(0)
plt.imshow(img, cmap ='jet')
plt.show()
print(img.shape)

In [ ]:
a = np.array([[[1,3,4]]])
print(a.shape)

In [ ]:
path = '/media/sansii/Software/san_projects/Major_project/KITTI_dataset/2015/training/disp_noc_0/'
img = cv2.imread(path+'000023_10.png',cv2.IMREAD_GRAYSCALE)
right_padding = np.zeros_like(np.array(img))

In [ ]:
right_padding = np.zeros((28,128 ,3))


In [46]:
def load_groudtruth_data(path):
    gt_path = sorted(glob.glob(path+'KITTI_dataset/2015/training/disp_noc_0/*.png'))
    for i in range(len(gt_path)):
        gt = np.array(cv2.imread(gt_path[i], cv2.IMREAD_GRAYSCALE))
        if gt.max() > 128:
            print("                    ",end = '\r')
            gt = disparity_threshold(gt, 128)
            print("Max:", gt.max(),end = '\r')
            cv2.imshow('gt',gt)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
            
        

In [47]:
path = '/media/sansii/Software/san_projects/Major_project/'
load_groudtruth_data(path)